In [ ]:
## train cv
## optimize all
## submit

## Importing libraries

In [54]:
import numpy as np
import pandas as pd
import os
import copy
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
import lightgbm as lgb
import xgboost as xgb
# import timeb
from collections import Counter
import datetime
from catboost import CatBoostRegressor, Pool
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit, RepeatedStratifiedKFold
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from bayes_opt import BayesianOptimization
import eli5
import shap
from IPython.display import HTML
import json
import altair as alt
from category_encoders.ordinal import OrdinalEncoder
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline
from typing import List

import os
import time
import datetime
import json
import gc
from numba import jit

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook

import lightgbm as lgb
import xgboost as xgb
from sklearn import metrics
from typing import Any
from itertools import product
import re
from tqdm import tqdm
from joblib import Parallel, delayed

random_seed = 33

In [55]:
# train = pd.read_csv('../data/preprocess/train895.csv')
# test = pd.read_csv('../data/preprocess/test895.csv')
# train = pd.read_csv('../data/preprocess/train_1225.csv')
# test = pd.read_csv('../data/preprocess/test_1225.csv')
train = pd.read_csv('../data/preprocess/train_1225_2.csv')
test = pd.read_csv('../data/preprocess/test_1225_2.csv')

## Helper functions and classes

In [56]:
from sklearn.metrics import cohen_kappa_score
def lgb_kappa_eval(y_true, y_pred):
    loss = cohen_kappa_score(y_true, y_pred,weights='quadratic')
    return "qwk", loss, True

def lgb_kappa_eval2(a1, a2):
    max_rat = 3
    a1 = np.asarray(a1, dtype=int)
    a2 = np.asarray(a2, dtype=int)

    hist1 = np.zeros((max_rat + 1, ))
    hist2 = np.zeros((max_rat + 1, ))

    o = 0
    for k in range(a1.shape[0]):
        i, j = a1[k], a2[k]
        hist1[i] += 1
        hist2[j] += 1
        o +=  (i - j) * (i - j)

    e = 0
    for i in range(max_rat + 1):
        for j in range(max_rat + 1):
            e += hist1[i] * hist2[j] * (i - j) * (i - j)

    e = e / a1.shape[0]

    return "qwk2", 1 - o / e, True

## https://github.com/catboost/tutorials/blob/master/custom_loss/custom_loss_and_metric_tutorial.ipynb
class catboost_qwk(object):
    def get_final_error(self, error, weight):
        return error / (weight + 1e-38)

    def is_max_optimal(self):
        return True

    def evaluate(self, approxes, target, weight):
        # approxes is list of indexed containers
        # (containers with only __len__ and __getitem__ defined), one container
        # per approx dimension. Each container contains floats.
        # weight is one dimensional indexed container.
        # target is float.   
        # weight parameter can be None.
        # Returns pair (error, weights sum)

        assert len(approxes) == 1
        assert len(target) == len(approxes[0])

        approx = approxes[0]

        error_sum = 0.0
        weight_sum = 1.0
        y_true = []
        y_pred = []
        for i in range(len(approx)):
            y_true.append(target[i])
            y_pred.append(approx[i])
        y_pred = np.array(y_pred)
        y_true = np.array(y_true)
        
        y_pred[y_pred <= 1.12232214] = 0
        y_pred[np.where(np.logical_and(y_pred > 1.12232214, y_pred <= 1.73925866))] = 1
        y_pred[np.where(np.logical_and(y_pred > 1.73925866, y_pred <= 2.22506454))] = 2
        y_pred[y_pred > 2.22506454] = 3
        
        error_sum = cohen_kappa_score(y_true, y_pred,weights='quadratic')
#         error_sum = qwk(y_true, y_pred)
        
        return error_sum, weight_sum

In [65]:
type(CatBoostRegressor())==CatBoostRegressor


True

In [57]:
from sklearn.base import BaseEstimator, TransformerMixin
@jit
def qwk(a1, a2):
    """
    Source: https://www.kaggle.com/c/data-science-bowl-2019/discussion/114133#latest-660168

    :param a1:
    :param a2:
    :param max_rat:
    :return:
    """
    max_rat = 3
    a1 = np.asarray(a1, dtype=int)
    a2 = np.asarray(a2, dtype=int)

    hist1 = np.zeros((max_rat + 1, ))
    hist2 = np.zeros((max_rat + 1, ))

    o = 0
    for k in range(a1.shape[0]):
        i, j = a1[k], a2[k]
        hist1[i] += 1
        hist2[j] += 1
        o +=  (i - j) * (i - j)

    e = 0
    for i in range(max_rat + 1):
        for j in range(max_rat + 1):
            e += hist1[i] * hist2[j] * (i - j) * (i - j)

    e = e / a1.shape[0]

    return 1 - o / e

def eval_qwk_lgb_regr(y_true, y_pred):
    """
    Fast cappa eval function for lgb.
    """
    y_pred[y_pred <= 1.12232214] = 0
    y_pred[np.where(np.logical_and(y_pred > 1.12232214, y_pred <= 1.73925866))] = 1
    y_pred[np.where(np.logical_and(y_pred > 1.73925866, y_pred <= 2.22506454))] = 2
    y_pred[y_pred > 2.22506454] = 3

    # y_pred = y_pred.reshape(len(np.unique(y_true)), -1).argmax(axis=0)

    return 'cappa', qwk(y_true, y_pred), True


class LGBWrapper_regr(object):
    """
    A wrapper for lightgbm model so that we will have a single api for various models.
    """

    def __init__(self):
        self.model = lgb.LGBMRegressor()

    def fit(self, X_train, y_train, X_valid=None, y_valid=None, X_holdout=None, y_holdout=None, params=None):
        eval_metric = eval_qwk_lgb_regr
        
        eval_set = [(X_train, y_train)]
        eval_names = ['train']
        self.model = self.model.set_params(**params)

        if X_valid is not None:
            eval_set.append((X_valid, y_valid))
            eval_names.append('valid')

        if X_holdout is not None:
            eval_set.append((X_holdout, y_holdout))
            eval_names.append('holdout')

        if 'cat_cols' in params.keys():
            cat_cols = [col for col in params['cat_cols'] if col in X_train.columns]
            if len(cat_cols) > 0:
                categorical_columns = params['cat_cols']
            else:
                categorical_columns = 'auto'
        else:
            categorical_columns = 'auto'

#                        eval_set=eval_set, eval_names=eval_names, eval_metric=lgb_kappa_eval,
        self.model.fit(X=X_train, y=y_train,
                       eval_set=eval_set, eval_names=eval_names, eval_metric=eval_metric,
                       verbose=params['verbose'], early_stopping_rounds=params['early_stopping_rounds'],
                       categorical_feature=categorical_columns)

        self.best_score_ = self.model.best_score_
        self.feature_importances_ = self.model.feature_importances_

    def predict(self, X_test):
        return self.model.predict(X_test, num_iteration=self.model.best_iteration_)


class CatboostWrapper_regr(object):
    """
    A wrapper for catboost model so that we will have a single api for various models.
    """

    def __init__(self):
        self.model = CatBoostRegressor()

    def fit(self, X_train, y_train, X_valid=None, y_valid=None, X_holdout=None, y_holdout=None, params=None):
        eval_set = []
        eval_names = []
        self.model = self.model.set_params(**params)
        self.model = self.model.set_params(eval_metric=catboost_qwk())
        
        
        if X_valid is not None:
            eval_set.append((X_valid, y_valid))
            eval_names.append('valid')
        elif X_holdout is not None:
            eval_set.append((X_holdout, y_holdout))
            eval_names.append('holdout')
        else:
            eval_set.append((X_train, y_train))
            eval_names.append('train')
        
        if 'cat_cols' in params.keys():
            cat_cols = [col for col in params['cat_cols'] if col in X_train.columns]
            if len(cat_cols) > 0:
                categorical_columns = params['cat_cols']
            else:
                categorical_columns = None
        else:
            categorical_columns = None

        self.model.fit(X=X_train, y=y_train,
                       eval_set=eval_set,
                       verbose=params['verbose'], early_stopping_rounds=params['early_stopping_rounds'],
                       cat_features=categorical_columns)

        self.best_score_ = self.model.get_best_score()
        
        test_pool=Pool(X_valid, y_valid,cat_features=categorical_columns)
        self.feature_importances_ = self.model.get_feature_importance(test_pool,thread_count=4)

    def predict(self, X_test):
        return self.model.predict(X_test, ntree_end=self.model.get_best_iteration())



class MainTransformer(BaseEstimator, TransformerMixin):

    def __init__(self, convert_cyclical: bool = False, create_interactions: bool = False, n_interactions: int = 20):
        """
        Main transformer for the data. Can be used for processing on the whole data.

        :param convert_cyclical: convert cyclical features into continuous
        :param create_interactions: create interactions between features
        """

        self.convert_cyclical = convert_cyclical
        self.create_interactions = create_interactions
        self.feats_for_interaction = None
        self.n_interactions = n_interactions

    def fit(self, X, y=None):

        if self.create_interactions:
            self.feats_for_interaction = [col for col in X.columns if 'sum' in col
                                          or 'mean' in col or 'max' in col or 'std' in col
                                          or 'attempt' in col]
            self.feats_for_interaction1 = np.random.choice(self.feats_for_interaction, self.n_interactions)
            self.feats_for_interaction2 = np.random.choice(self.feats_for_interaction, self.n_interactions)

        return self

    def transform(self, X, y=None):
        data = copy.deepcopy(X)
        if self.create_interactions:
            for col1 in self.feats_for_interaction1:
                for col2 in self.feats_for_interaction2:
                    data[f'{col1}_int_{col2}'] = data[col1] * data[col2]

        if self.convert_cyclical:
            data['timestampHour'] = np.sin(2 * np.pi * data['timestampHour'] / 23.0)
            data['timestampMonth'] = np.sin(2 * np.pi * data['timestampMonth'] / 23.0)
            data['timestampWeek'] = np.sin(2 * np.pi * data['timestampWeek'] / 23.0)
            data['timestampMinute'] = np.sin(2 * np.pi * data['timestampMinute'] / 23.0)

#         data['installation_session_count'] = data.groupby(['installation_id'])['Clip'].transform('count')
#         data['installation_duration_mean'] = data.groupby(['installation_id'])['duration_mean'].transform('mean')
#         data['installation_title_nunique'] = data.groupby(['installation_id'])['session_title'].transform('nunique')

#         data['sum_event_code_count'] = data[['2000', '3010', '3110', '4070', '4090', '4030', '4035', '4021', '4020', '4010', '2080', '2083', '2040', '2020', '2030', '3021', '3121', '2050', '3020', '3120', '2060', '2070', '4031', '4025', '5000', '5010', '2081', '2025', '4022', '2035', '4040', '4100', '2010', '4110', '4045', '4095', '4220', '2075', '4230', '4235', '4080', '4050']].sum(axis=1)

        # data['installation_event_code_count_mean'] = data.groupby(['installation_id'])['sum_event_code_count'].transform('mean')

        return data

    def fit_transform(self, X, y=None, **fit_params):
        data = copy.deepcopy(X)
        self.fit(data)
        return self.transform(data)


class FeatureTransformer(BaseEstimator, TransformerMixin):

    def __init__(self, main_cat_features: list = None, num_cols: list = None):
        """

        :param main_cat_features:
        :param num_cols:
        """
        self.main_cat_features = main_cat_features
        self.num_cols = num_cols

    def fit(self, X, y=None):

#         self.num_cols = [col for col in X.columns if 'sum' in col or 'mean' in col or 'max' in col or 'std' in col
#                          or 'attempt' in col]
        

        return self

    def transform(self, X, y=None):
        data = copy.deepcopy(X)
#         for col in self.num_cols:
#             data[f'{col}_to_mean'] = data[col] / data.groupby('installation_id')[col].transform('mean')
#             data[f'{col}_to_std'] = data[col] / data.groupby('installation_id')[col].transform('std')

        return data

    def fit_transform(self, X, y=None, **fit_params):
        data = copy.deepcopy(X)
        self.fit(data)
        return self.transform(data)

In [58]:
class RegressorModel(object):
    """
    A wrapper class for regression models.
    It can be used for training and prediction.
    Can plot feature importance and training progress (if relevant for model).
    """

    def __init__(self, columns: list=None, model_wrapper=None, random_state=33):
        """
        :param columns: columns to train
        :param model_wrapper:
        """
        self.columns = columns
        self.model_wrapper = model_wrapper
        self.random_seed = random_state
        
        self.result_dict = {}
        self.train_one_fold = False
        self.preprocesser = None

    def fit(self, X: pd.DataFrame, y,
            X_holdout: pd.DataFrame = None, y_holdout=None,
            folds=None,
            params: dict = None,
            eval_metric='rmse',
            cols_to_drop: list = None,
            preprocesser=None,
            transformers: dict = None,
            adversarial: bool = False,
            plot: bool = True):
        """
        Training the model.

        :param X: training data
        :param y: training target
        :param X_holdout: holdout data
        :param y_holdout: holdout target
        :param folds: folds to split the data. If not defined, then model will be trained on the whole X
        :param params: training parameters
        :param eval_metric: metric for validataion
        :param cols_to_drop: list of columns to drop (for example ID)
        :param preprocesser: preprocesser class
        :param transformers: transformer to use on folds
        :param adversarial
        :return:
        """
# transformers,n_target,oof,

        if folds is None:
            folds = KFold(n_splits=3, random_state=self.random_seed)
            self.train_one_fold = True

        self.columns = X.columns if self.columns is None else self.columns
        self.feature_importances = pd.DataFrame(columns=['feature', 'importance'])
        self.trained_transformers = {k: [] for k in transformers}
        self.transformers = transformers
        self.models = []
        self.folds_dict = {}
        self.eval_metric = eval_metric
        n_target = 1
        self.oof = np.zeros((len(X), n_target))
        self.n_target = n_target

        X = X[self.columns]
        if X_holdout is not None:
            X_holdout = X_holdout[self.columns]

        if preprocesser is not None:
            self.preprocesser = preprocesser
            self.preprocesser.fit(X, y)
            X = self.preprocesser.transform(X, y)
            self.columns = X.columns.tolist()
            if X_holdout is not None:
                X_holdout = self.preprocesser.transform(X_holdout)

        for fold_n, (train_index, valid_index) in enumerate(folds.split(X, y, X['installation_id'])):

            if X_holdout is not None:
                X_hold = X_holdout.copy()
            else:
                X_hold = None
            self.folds_dict[fold_n] = {}
            if params['verbose']:
                print(f'Fold {fold_n + 1} started at {time.ctime()}')
#             self.folds_dict[fold_n] = {}

            X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
            y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
            if self.train_one_fold:
                X_train = X[self.original_columns]
                y_train = y
                X_valid = None
                y_valid = None

            datasets = {'X_train': X_train, 'X_valid': X_valid, 'X_holdout': X_hold, 'y_train': y_train}
            X_train, X_valid, X_hold = self.transform_(datasets, cols_to_drop)

            self.folds_dict[fold_n]['columns'] = X_train.columns.tolist()

            model = copy.deepcopy(self.model_wrapper)

            if adversarial:
                X_new1 = X_train.copy()
                if X_valid is not None:
                    X_new2 = X_valid.copy()
                elif X_holdout is not None:
                    X_new2 = X_holdout.copy()
                X_new = pd.concat([X_new1, X_new2], axis=0)
                y_new = np.hstack((np.zeros((X_new1.shape[0])), np.ones((X_new2.shape[0]))))
                X_train, X_valid, y_train, y_valid = train_test_split(X_new, y_new)

            model.fit(X_train, y_train, X_valid, y_valid, X_hold, y_holdout, params=params)

            self.folds_dict[fold_n]['scores'] = model.best_score_
            if self.oof.shape[0] != len(X):
                self.oof = np.zeros((X.shape[0], self.oof.shape[1]))
            if not adversarial:
                self.oof[valid_index] = model.predict(X_valid).reshape(-1, n_target)

            fold_importance = pd.DataFrame(list(zip(X_train.columns, model.feature_importances_)),
                                           columns=['feature', 'importance'])
            self.feature_importances = self.feature_importances.append(fold_importance)
            self.models.append(model)

        self.feature_importances['importance'] = self.feature_importances['importance'].astype(int)

        # if params['verbose']:
        self.calc_scores_()

        if plot:
            # print(classification_report(y, self.oof.argmax(1)))
            fig, ax = plt.subplots(figsize=(16, 12))
            plt.subplot(2, 2, 1)
            self.plot_feature_importance(top_n=20)
            plt.subplot(2, 2, 2)
            self.plot_metric()
            plt.subplot(2, 2, 3)
            plt.hist(y.values.reshape(-1, 1) - self.oof)
            plt.title('Distribution of errors')
            plt.subplot(2, 2, 4)
            plt.hist(self.oof)
            plt.title('Distribution of oof predictions');

    def transform_(self, datasets, cols_to_drop):
        for name, transformer in self.transformers.items():
            transformer.fit(datasets['X_train'], datasets['y_train'])
            datasets['X_train'] = transformer.transform(datasets['X_train'])
            if datasets['X_valid'] is not None:
                datasets['X_valid'] = transformer.transform(datasets['X_valid'])
            if datasets['X_holdout'] is not None:
                datasets['X_holdout'] = transformer.transform(datasets['X_holdout'])
            self.trained_transformers[name].append(transformer)
        if cols_to_drop is not None:
            cols_to_drop = [col for col in cols_to_drop if col in datasets['X_train'].columns]

            datasets['X_train'] = datasets['X_train'].drop(cols_to_drop, axis=1)
            if datasets['X_valid'] is not None:
                datasets['X_valid'] = datasets['X_valid'].drop(cols_to_drop, axis=1)
            if datasets['X_holdout'] is not None:
                datasets['X_holdout'] = datasets['X_holdout'].drop(cols_to_drop, axis=1)
        self.cols_to_drop = cols_to_drop

        return datasets['X_train'], datasets['X_valid'], datasets['X_holdout']

    def calc_scores_(self):
        print()
        datasets = [k for k, v in [v['scores'] for k, v in self.folds_dict.items()][0].items() if len(v) > 0]
        self.scores = {}
        for d in datasets:
            scores = [v['scores'][d][self.eval_metric] for k, v in self.folds_dict.items()]
            print(f"CV mean score on {d}: {np.mean(scores):.4f} +/- {np.std(scores):.4f} std.")
            self.scores[d] = np.mean(scores)

    def predict(self, X_test, averaging: str = 'usual'):
        """
        Make prediction

        :param X_test:
        :param averaging: method of averaging
        :return:
        """
        full_prediction = np.zeros((X_test.shape[0], self.oof.shape[1]))
        if self.preprocesser is not None:
            X_test = self.preprocesser.transform(X_test)
        for i in range(len(self.models)):
            X_t = X_test.copy()
            for name, transformers in self.trained_transformers.items():
                X_t = transformers[i].transform(X_t)

            if self.cols_to_drop is not None:
                cols_to_drop = [col for col in self.cols_to_drop if col in X_t.columns]
                X_t = X_t.drop(cols_to_drop, axis=1)
            y_pred = self.models[i].predict(X_t[self.folds_dict[i]['columns']]).reshape(-1, full_prediction.shape[1])

            # if case transformation changes the number of the rows
            if full_prediction.shape[0] != len(y_pred):
                full_prediction = np.zeros((y_pred.shape[0], self.oof.shape[1]))

            if averaging == 'usual':
                full_prediction += y_pred
            elif averaging == 'rank':
                full_prediction += pd.Series(y_pred).rank().values

        return full_prediction / len(self.models)

    def plot_feature_importance(self, drop_null_importance: bool = True, top_n: int = 10):
        """
        Plot default feature importance.

        :param drop_null_importance: drop columns with null feature importance
        :param top_n: show top n columns
        :return:
        """

        top_feats = self.get_top_features(drop_null_importance, top_n)
        feature_importances = self.feature_importances.loc[self.feature_importances['feature'].isin(top_feats)]
        feature_importances['feature'] = feature_importances['feature'].astype(str)
        top_feats = [str(i) for i in top_feats]
        sns.barplot(data=feature_importances, x='importance', y='feature', orient='h', order=top_feats)
        plt.title('Feature importances')
        print(feature_importances)

    def get_top_features(self, drop_null_importance: bool = True, top_n: int = 10):
        """
        Get top features by importance.

        :param drop_null_importance:
        :param top_n:
        :return:
        """
        grouped_feats = self.feature_importances.groupby(['feature'])['importance'].mean()
        if drop_null_importance:
            grouped_feats = grouped_feats[grouped_feats != 0]
        return list(grouped_feats.sort_values(ascending=False).index)[:top_n]

    def plot_metric(self):
        """
        Plot training progress.
        Inspired by `plot_metric` from https://lightgbm.readthedocs.io/en/latest/_modules/lightgbm/plotting.html

        :return:
        """
        full_evals_results = pd.DataFrame()
        for model in self.models:
            evals_result = pd.DataFrame()
            for k in model.model.evals_result_.keys():
                evals_result[k] = model.model.evals_result_[k][self.eval_metric]
            evals_result = evals_result.reset_index().rename(columns={'index': 'iteration'})
            full_evals_results = full_evals_results.append(evals_result)

        full_evals_results = full_evals_results.melt(id_vars=['iteration']).rename(columns={'value': self.eval_metric,
                                                                                            'variable': 'dataset'})
        sns.lineplot(data=full_evals_results, x='iteration', y=self.eval_metric, hue='dataset')
        plt.title('Training progress')


In [59]:
lgb_params = {'n_estimators':2000,
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': 'rmse',
            'subsample': 0.75,
            'subsample_freq': 1,
            'learning_rate': 0.04,
            'feature_fraction': 0.9,
         'max_depth': 15,
            'lambda_l1': 1,  
            'lambda_l2': 1,
            'verbose': 100,
            'early_stopping_rounds': 100, 
#             'eval_metric': lgb_kappa_eval,
            'eval_metric': 'cappa'
            }

cat_params = {'iterations':2000,
            'loss_function': 'RMSE',
            'verbose': 10,
            'early_stopping_rounds': 100, 
#             'task_type':"GPU",
#             'devices':'0:1',
            }
para_tune = {'feature_fraction': 0.9985706623844444, 'lambda_l1': 1.3464702887971034, 'lambda_l2': 24.03486401669668, 'max_depth': 9, 'num_leaves': 20, 'subsample': 0.9982720526777418, 'subsample_freq': 2}
for k in para_tune:
    lgb_params[k] = para_tune[k]

In [61]:
y = train['accuracy_group']
n_fold = 5
folds = GroupKFold(n_splits=n_fold)

cat_cols = ['title','world','timestamp_weekday']
cols_to_drop = ['installation_id', 'game_session', 'timestamp', 'accuracy_group']
lgb_params['cat_cols'] = cat_cols']
cat_params['cat_cols'] = cat_cols

mt = MainTransformer()
ft = FeatureTransformer()
transformers = {'ft': ft}
regressor_model1 = RegressorModel(model_wrapper=LGBWrapper_regr())
regressor_model1.fit(X=train, y=y, folds=folds, params=lgb_params, preprocesser=mt, transformers=transformers,
                    eval_metric='cappa', cols_to_drop=cols_to_drop)
# regressor_model1 = RegressorModel(model_wrapper=CatboostWrapper_regr())
# regressor_model1.fit(X=train, y=y, folds=folds, params=cat_params, preprocesser=mt, transformers=transformers,
#                     eval_metric='cappa', cols_to_drop=cols_to_drop)

Fold 1 started at Wed Jan  1 08:21:17 2020
0:	learn: 0.0000000	test: 0.0000000	best: 0.0000000 (0)	total: 234ms	remaining: 7m 48s
10:	learn: 0.3014725	test: 0.3057982	best: 0.3057982 (10)	total: 2.56s	remaining: 7m 43s
20:	learn: 0.3289608	test: 0.3370217	best: 0.3370217 (20)	total: 4.86s	remaining: 7m 37s
30:	learn: 0.4518421	test: 0.4640883	best: 0.4640883 (30)	total: 7.11s	remaining: 7m 31s
40:	learn: 0.5073477	test: 0.5136931	best: 0.5136931 (40)	total: 9.32s	remaining: 7m 25s
50:	learn: 0.5403683	test: 0.5418318	best: 0.5418318 (50)	total: 11.6s	remaining: 7m 21s
60:	learn: 0.5569497	test: 0.5592128	best: 0.5592128 (60)	total: 13.8s	remaining: 7m 18s
70:	learn: 0.5666205	test: 0.5642680	best: 0.5648081 (69)	total: 16s	remaining: 7m 14s
80:	learn: 0.5752564	test: 0.5764790	best: 0.5764790 (80)	total: 18.2s	remaining: 7m 12s
90:	learn: 0.5830643	test: 0.5849775	best: 0.5849775 (90)	total: 20.5s	remaining: 7m 10s
100:	learn: 0.5904211	test: 0.5939415	best: 0.5939415 (100)	total: 22.8

350:	learn: 0.6268189	test: 0.6055652	best: 0.6080276 (335)	total: 1m 16s	remaining: 6m 1s
360:	learn: 0.6270134	test: 0.6065461	best: 0.6080276 (335)	total: 1m 19s	remaining: 5m 58s
370:	learn: 0.6275812	test: 0.6057796	best: 0.6080276 (335)	total: 1m 21s	remaining: 5m 56s
380:	learn: 0.6283412	test: 0.6063220	best: 0.6080276 (335)	total: 1m 23s	remaining: 5m 54s
390:	learn: 0.6293357	test: 0.6063312	best: 0.6080276 (335)	total: 1m 25s	remaining: 5m 52s
400:	learn: 0.6299314	test: 0.6060244	best: 0.6080276 (335)	total: 1m 27s	remaining: 5m 49s
410:	learn: 0.6312306	test: 0.6053983	best: 0.6080276 (335)	total: 1m 29s	remaining: 5m 47s
420:	learn: 0.6314263	test: 0.6048967	best: 0.6080276 (335)	total: 1m 32s	remaining: 5m 45s
430:	learn: 0.6320067	test: 0.6047865	best: 0.6080276 (335)	total: 1m 34s	remaining: 5m 43s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6080275653
bestIteration = 335

Shrink model to first 336 iterations.
Fold 3 started at Wed Jan  1 08:24

40:	learn: 0.5241999	test: 0.4858684	best: 0.4858684 (40)	total: 9.62s	remaining: 7m 39s
50:	learn: 0.5471454	test: 0.5061421	best: 0.5070935 (49)	total: 12s	remaining: 7m 38s
60:	learn: 0.5642502	test: 0.5211445	best: 0.5211445 (60)	total: 14.4s	remaining: 7m 37s
70:	learn: 0.5768999	test: 0.5345700	best: 0.5345700 (70)	total: 16.8s	remaining: 7m 36s
80:	learn: 0.5874234	test: 0.5434719	best: 0.5434719 (80)	total: 19.2s	remaining: 7m 35s
90:	learn: 0.5957483	test: 0.5493386	best: 0.5493433 (89)	total: 21.6s	remaining: 7m 32s
100:	learn: 0.6010213	test: 0.5516960	best: 0.5516960 (100)	total: 24s	remaining: 7m 31s
110:	learn: 0.6059419	test: 0.5565484	best: 0.5569113 (109)	total: 26.4s	remaining: 7m 29s
120:	learn: 0.6104480	test: 0.5617318	best: 0.5617318 (120)	total: 28.8s	remaining: 7m 27s
130:	learn: 0.6149740	test: 0.5619935	best: 0.5628585 (121)	total: 31.2s	remaining: 7m 25s
140:	learn: 0.6165149	test: 0.5632380	best: 0.5647282 (139)	total: 33.6s	remaining: 7m 23s
150:	learn: 0.6

KeyError: 'cappa'

整理別人kernel中使用過的特徵

feature selection
* lasso?
* 相對訊息?

bayisan optimization參數

思考有哪些訓練方式:
有沒有比k-fold更好的?
semi-supervised learning? 將沒有assessment的使用者label加進訓練資料中
k-fold optimize th驗證結果

訓練regressor
optimize th


應該K fold predict reduce train
先用Bayesian優化predict kappa
再用K fold結果來optimize 邊界

用LGB,XGB,Catboost嘗試regressor結果


## Making predictions

The preprocessing is a class, which was initially written by Abhishek Thakur here: https://www.kaggle.com/c/petfinder-adoption-prediction/discussion/76107 and later improved here https://www.kaggle.com/naveenasaithambi/optimizedrounder-improved (the improvement is is speed).

It can be used to find optimal coefficients for thresholds. In this kernel I'll show an example, but when you do it, don't forget a proper validation.

In [ ]:
from functools import partial
import scipy as sp
class OptimizedRounder(object):
    """
    An optimizer for rounding thresholds
    to maximize Quadratic Weighted Kappa (QWK) score
    # https://www.kaggle.com/naveenasaithambi/optimizedrounder-improved
    """
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        """
        Get loss according to
        using current coefficients
        
        :param coef: A list of coefficients that will be used for rounding
        :param X: The raw predictions
        :param y: The ground truth labels
        """
        X_p = pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])

        return -qwk(y, X_p)

    def fit(self, X, y):
        """
        Optimize rounding thresholds
        
        :param X: The raw predictions
        :param y: The ground truth labels
        """
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')

    def predict(self, X, coef):
        """
        Make predictions with specified thresholds
        
        :param X: The raw predictions
        :param coef: A list of coefficients that will be used for rounding
        """
        return pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])


    def coefficients(self):
        """
        Return the optimized coefficients
        """
        return self.coef_['x']

In [ ]:
pr1 = regressor_model1.predict(reduce_train)

optR = OptimizedRounder()
optR.fit(pr1.reshape(-1,), y)
coefficients = optR.coefficients()

In [ ]:
opt_preds = optR.predict(pr1.reshape(-1, ), coefficients)
qwk(y, opt_preds)

In [ ]:
def cut_pred(pr1: np.array, coef: list):
    assert len(coef)==3
    pr1[pr1 <= coef[0]] = 0
    pr1[np.where(np.logical_and(pr1 > coef[0], pr1 <= coef[1]))] = 1
    pr1[np.where(np.logical_and(pr1 > coef[1], pr1 <= coef[2]))] = 2
    pr1[pr1 > coef[2]] = 3
    
    return pr1

pr1 = cut_pred(pr1,coef)

In [ ]:
def make_submission(sample_submission, pred, submit_file_name='submission.csv'):
    sample_submission['accuracy_group'] = pred.astype(int)
    sample_submission.to_csv(submit_file_name, index=False)
    print(sample_submission['accuracy_group'].value_counts(normalize=True))
    
make_submission(sample_submission=sample_submission, pred=pr1, submit_file_name='submission.csv')